In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install -U openai-whisper
pip install nlptutti

In [ ]:
audio_path = "/content/drive/MyDrive/인강프로젝트/combined_audio/최은진/01강 확률적 사고_combined_audio.wav"

In [ ]:
smi_path = "/content/drive/MyDrive/인강프로젝트/분석/정확도/최은진/[2022 수능특강] 최은진의 확률과 통계/01강 확률적 사고_PRE 수능특강_(고3-발전).smi"

# STT (whisper)

In [ ]:
import whisper

model = whisper.load_model("medium")
result = model.transcribe(audio_path)
hypothesis = result["text"]

# smi -> txt

In [ ]:
import re # smi 파일에서 텍스트 뽑아내는 모듈
import pandas as pd
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

def smi_to_df (smi_path):
    file = open(smi_path, 'r', encoding = 'cp949')
    wholefile = file.readlines()
    # 모든 대문자를 소문자로 변환
    wholefile_lower = [''.join(x.lower() for x in s) for s in wholefile]
    # 타임스템프와 본문 분리
    pattern = re.compile(r'<sync start=(\d+)><p class=krcc>\n(.*?)\n', re.DOTALL)

    matches = pattern.findall(''.join(wholefile_lower))

    df = pd.DataFrame(matches, columns=['Time_Stamp(s)', 'Text'])
    df['Time_Stamp(s)'] = df['Time_Stamp(s)'].apply(lambda x: float(x)/1000)

    # Text를 하나의 문장으로 합치기
    text = ''.join(df['Text'].tolist())
    sentences = sent_tokenize(''.join(text))
    nltk_df = pd.DataFrame(sentences, columns=['Text'])

    return df, nltk_df

In [ ]:
df, nltk_df = smi_to_df(smi_path)
reference = ' '.join(nltk_df['Text'])

# 강의명, 강좌명 추출

In [3]:
import os
import pandas as pd
# 작업 디렉토리 설정
directory = '/content/drive/MyDrive/인강프로젝트/분석/정확도/최은진/[2022 수능특강] 최은진의 확률과 통계'

# 디렉토리 내 파일 목록 가져오기
files = os.listdir(directory)

# 파일들을 순회하면서 폴더명을 컬럼1로, 파일명의 _ 앞부분을 컬럼2로 추가
for file in files:
    if file[-4:] == '.smi':
      folder_name = os.path.basename(directory)  # 폴더명 추출
      file_name = os.path.splitext(file)[0]  # 확장자 제거

# CER

In [ ]:
import nlptutti as metrics

result = metrics.get_cer(reference, hypothesis, rm_punctuation=True)
cer = result['cer']
substitutions = result['substitutions']
deletions = result['deletions']
insertions = result['insertions']
# prints: [cer, substitutions, deletions, insertions] -> [CER = 0 / 34, S = 0, D = 0, I = 0]

In [ ]:
print([cer, substitutions, deletions, insertions])

In [ ]:
# result를 dataframe으로 변환
data = [[folder_name, file_name, cer, substitutions, deletions, insertions]]
df = pd.DataFrame(data, columns=['강좌명', '강의명', 'CER', 'substitutions', 'deletions', 'insertions'])

# 시각화

In [ ]:
import plotly.figure_factory as ff

#테이블 생성
fig =  ff.create_table(df.iloc[:, 2:])

fig.show()